In [2]:
import spacy
import os
import numpy as np

# Загрузка русской языковой модели
nlp = spacy.load("ru_core_news_md")

# Список стоп-слов для удаления
stop_words = nlp.Defaults.stop_words

# Функция для удаления стоп-слов из текста
def remove_stop_words(text):
    doc = nlp(text)
    filtered_tokens = [token.text for token in doc if token.text.lower() not in stop_words]
    return " ".join(filtered_tokens)

# Функция для обработки файлов
def process_files(directory):
    sentence_data = []
    for file_name in os.listdir(directory):
        if file_name.endswith(".txt"):
            file_path = os.path.join(directory, file_name)
            with open(file_path, 'r', encoding='utf-8') as file:
                content = file.read()
            
            # Разбиваем текст на предложения
            doc = nlp(content)
            sentences = [sent.text.strip() for sent in doc.sents]
            
            # Обрабатываем каждое предложение
            for sent_idx, sentence in enumerate(sentences):
                # Удаляем стоп-слова
                filtered_sentence = remove_stop_words(sentence)
                
                # Проверяем, что предложение не пустое
                if not filtered_sentence.strip():
                    print(f"Пустое предложение после удаления стоп-слов: {sentence}")
                    continue
                
                # Векторизуем предложение
                vector = nlp(filtered_sentence).vector
                
                # Проверяем, что вектор не нулевой
                if np.all(vector == 0):
                    print(f"Нулевой вектор для предложения: {filtered_sentence}")
                    continue
                
                # Сохраняем данные
                sentence_data.append({
                    "document_id": file_name,
                    "sentence_id": sent_idx + 1,
                    "vector": vector,
                    "original_sentence": sentence,
                    "filtered_sentence": filtered_sentence
                })
    
    return sentence_data

# Функция для вычисления косинусного сходства
def cosine_similarity(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm1 = np.linalg.norm(vector1)
    norm2 = np.linalg.norm(vector2)
    
    # Проверка на нулевую норму
    if norm1 == 0 or norm2 == 0:
        return 0.0
    
    return dot_product / (norm1 * norm2)

# Функция для поиска ближайшего предложения
def find_closest_sentence(target_vector, sentence_data):
    max_similarity = -1
    closest_sentence = None
    
    for data in sentence_data:
        similarity = cosine_similarity(target_vector, data["vector"])
        if similarity > max_similarity:
            max_similarity = similarity
            closest_sentence = data
    
    return closest_sentence, max_similarity

# Основной код
if __name__ == "__main__":
    # Укажите путь к директории с текстовыми файлами
    directory_path = "./"
    
    # Обрабатываем файлы
    sentence_data = process_files(directory_path)
    
    # Формируем поисковый запрос
    search_query = "искомое предложение также отфильтровать на стоп-слова"  # Замените на текст из одного предложения документа
    search_query_filtered = remove_stop_words(search_query)
    search_query_vector = nlp(search_query_filtered).vector
    
    # Ищем ближайшее предложение для оригинального поискового запроса
    closest_sentence, similarity = find_closest_sentence(search_query_vector, sentence_data)
    
    # Вывод результатов для оригинального поискового запроса
    print("Поисковый запрос (оригинал):", search_query)
    print("Ближайшее предложение найдено в документе:", closest_sentence["document_id"])
    print("Идентификатор предложения:", closest_sentence["sentence_id"])
    print("Оригинальное предложение:", closest_sentence["original_sentence"])
    print("Отфильтрованное предложение:", closest_sentence["filtered_sentence"])
    print("Сходство:", similarity)
    
    # Ищем ближайшее предложение для отфильтрованного поискового запроса
    closest_filtered_sentence, filtered_similarity = find_closest_sentence(search_query_vector, sentence_data)
    
    # Вывод результатов для отфильтрованного поискового запроса
    print("\nПоисковый запрос (отфильтрованный):", search_query_filtered)
    print("Ближайшее предложение найдено в документе:", closest_filtered_sentence["document_id"])
    print("Идентификатор предложения:", closest_filtered_sentence["sentence_id"])
    print("Оригинальное предложение:", closest_filtered_sentence["original_sentence"])
    print("Отфильтрованное предложение:", closest_filtered_sentence["filtered_sentence"])
    print("Сходство:", filtered_similarity)

Поисковый запрос (оригинал): искомое предложение также отфильтровать на стоп-слова
Ближайшее предложение найдено в документе: file4.txt
Идентификатор предложения: 1
Оригинальное предложение: искомое предложение также отфильтровать на стоп-слова
Отфильтрованное предложение: искомое предложение отфильтровать стоп - слова
Сходство: 0.99999994

Поисковый запрос (отфильтрованный): искомое предложение отфильтровать стоп - слова
Ближайшее предложение найдено в документе: file4.txt
Идентификатор предложения: 1
Оригинальное предложение: искомое предложение также отфильтровать на стоп-слова
Отфильтрованное предложение: искомое предложение отфильтровать стоп - слова
Сходство: 0.99999994
